# Artificial Neural Network

### Importing the libraries

In [207]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [208]:
torch.__version__

'2.6.0+cu124'

## Part 1 - Data Preprocessing

### Importing the dataset

In [209]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values # write down here why we are using 3:-1 why these columns
y = dataset.iloc[:, -1].values

We used dataset.iloc[ : , 3 : -1 ] to select the columns from 3rd index upto the last except the last column as the last column consist of the y value and the first three columns bing name creditScore and index are not needed to train the model

In [210]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [211]:
# from google.colab import drive
# drive.mount('/content/drive')

In [212]:
print(X[:5])

[[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]
 [502 'France' 'Female' 42 8 159660.8 3 1 0 113931.57]
 [699 'France' 'Female' 39 1 0.0 2 0 0 93826.63]
 [850 'Spain' 'Female' 43 2 125510.82 1 1 1 79084.1]]


In [213]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [214]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X[:, 2] = X[:, 2].astype(str)
X[:, 2] = le.fit_transform(X[:, 2])

In [215]:
print(X[:8])

[[619 'France' 0 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 0 41 1 83807.86 1 0 1 112542.58]
 [502 'France' 0 42 8 159660.8 3 1 0 113931.57]
 [699 'France' 0 39 1 0.0 2 0 0 93826.63]
 [850 'Spain' 0 43 2 125510.82 1 1 1 79084.1]
 [645 'Spain' 1 44 8 113755.78 2 1 0 149756.71]
 [822 'France' 1 50 7 0.0 2 1 1 10062.8]
 [376 'Germany' 0 29 4 115046.74 4 1 0 119346.88]]


One Hot Encoding the "Geography" column

In [216]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X=ct.fit_transform(X)

In [217]:
print(X[:5])

[[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]
 [0.0 0.0 1.0 608 0 41 1 83807.86 1 0 1 112542.58]
 [1.0 0.0 0.0 502 0 42 8 159660.8 3 1 0 113931.57]
 [1.0 0.0 0.0 699 0 39 1 0.0 2 0 0 93826.63]
 [0.0 0.0 1.0 850 0 43 2 125510.82 1 1 1 79084.1]]


### Splitting the dataset into the Training set and Test set

In [218]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [219]:
X_train.shape , y_train.shape , X_test.shape , y_test.shape

((8000, 12), (8000,), (2000, 12), (2000,))

### Feature Scaling

In [220]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

In [221]:
X_train.shape

(8000, 12)

<h1>Device Agnostic Code</h1>

In [222]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [223]:
X_train=torch.tensor(X_train,dtype=torch.float32).to(device)
X_test=torch.tensor(X_test,dtype=torch.float32).to(device)
y_train=torch.tensor(y_train,dtype=torch.float32).to(device)
y_test=torch.tensor(y_test,dtype=torch.float32).to(device)

## Part 2 - Building the ANN

### Initializing the ANN

In [224]:
torch.manual_seed(1)
class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(12,6),
            nn.ReLU(),
            nn.Linear(6,6),
            nn.ReLU(),
            nn.Linear(6,1),
            nn.Sigmoid()
        )

    def forward(self,x):
        return self.layers(x)

In [225]:
model=ANN()

In [226]:
model.to(device)

ANN(
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=6, bias=True)
    (1): ReLU()
    (2): Linear(in_features=6, out_features=6, bias=True)
    (3): ReLU()
    (4): Linear(in_features=6, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

### Adding the input layer and the first hidden layer

### Adding the second hidden layer

### Adding the output layer

## Part 3 - Training the ANN

### Compiling the ANN

In [227]:
loss= nn.BCELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.1)


In [228]:
from sklearn.metrics import accuracy_score

### Training the ANN on the Training set

In [229]:
epoch=100
for i in range(epoch):
    #training loop
    y_pred=model(X_train)
    loss_value=loss(y_pred,y_train.reshape(-1,1))
    optimizer.zero_grad()
    y_pred=model(torch.tensor(X_train,dtype=torch.float32))
    loss_value.backward()
    optimizer.step()

    #testing loop
    model.eval()
    with torch.inference_mode():
        y_pred=model(torch.tensor(X_test,dtype=torch.float32))
        y_pred=y_pred.round()
        if i%10==0:
            print("Epoch",i,"Loss",loss_value)
            print("Test Accurancy",accuracy_score(y_test,y_pred))


Epoch 0 Loss tensor(0.5992, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.7925
Epoch 10 Loss tensor(0.4465, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.7925


<ipython-input-229-aa273e145c23>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_train,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_test,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_train,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().d

Epoch 20 Loss tensor(0.4351, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.7925
Epoch 30 Loss tensor(0.4225, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.7925
Epoch 40 Loss tensor(0.3861, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.8135
Epoch 50 Loss tensor(0.3673, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.8375


<ipython-input-229-aa273e145c23>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_train,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_test,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_train,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().d

Epoch 60 Loss tensor(0.3678, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.846
Epoch 70 Loss tensor(0.3552, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.854
Epoch 80 Loss tensor(0.3483, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.86
Epoch 90 Loss tensor(0.3443, grad_fn=<BinaryCrossEntropyBackward0>)
Test Accurancy 0.8585


<ipython-input-229-aa273e145c23>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_train,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_test,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred=model(torch.tensor(X_train,dtype=torch.float32))
<ipython-input-229-aa273e145c23>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().d

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Extra**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

**Solution**

In [230]:
[[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]

[[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]

### Predicting the Test set results

In [231]:
y_pred = model(X_test)

In [232]:
y_pred=y_pred.round().detach().cpu().numpy()

### Making the Confusion Matrix

In [233]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.86
[[1521   64]
 [ 216  199]]


write down about precision recall f1-score, why is it better than just accuracy, what are some other interesting metrics u can find